In [1]:
from utils import (
    download_file,
    parsear_archivo,
    random_path,
    calcular_distancias,
    cumple_capacidad,
    dist
)
from tqdm.notebook import tqdm as tqdm
import math

# Descargo y parseo datos

In [2]:
download_file("https://modelosuno.okapii.com/Problemas/problema2022/problema_dos.txt", "problema_dos.txt", override=True)

  0%|          | 0.00/470k [00:00<?, ?iB/s]

In [3]:
CAPACIDAD, DIMENSION, DEMANDAS, TIPO_ARISTA, COORDS = parsear_archivo("problema_dos.txt")

In [4]:
DATA = [
    (k, v, COORDS[k]["x"], COORDS[k]["y"]) for k,v in DEMANDAS.items()
]
COORDS_S = {
    k: (v["x"], v["y"]) for k, v in COORDS.items()
}

# Armo recorridos

In [5]:
class LoopError(Exception):
    pass

In [15]:
import multiprocessing as mp

def generar_camino(initial, demandas, data, dimension, coords_s, capacidad):
    try:
        if demandas[initial] < 0:
            return None
        seq = [initial]
        seqset = set(seq)
        capacities = [demandas[initial]]

        for _ in range(dimension - 1):
            # while len(seq) < DIMENSION:
            last = seq[-1]
            last_capacity = capacities[-1]
            x0, y0 = coords_s[last]

            best_candidate = None
            best_distance = math.inf

            for k, dem, x1, y1 in data:
                if k in seqset:
                    continue
                if not 0 <= last_capacity + dem <= capacidad:
                    continue
                d = (x0 - x1) ** 2 + (y0 - y1) ** 2
                if d < best_distance:
                    best_candidate = k
                    best_distance = d

            if best_candidate is None:
                raise LoopError

            seq.append(best_candidate)
            seqset.add(best_candidate)
            capacities.append(last_capacity + demandas[best_candidate])

        return seq
    except LoopError:
        return None

import functools

initial_candidates = list(range(1,201))
costos = []
caminos = []

with mp.Pool(16) as p, tqdm(total=len(initial_candidates)) as bar:
    f = functools.partial(
        generar_camino,
        demandas=DEMANDAS,
        data=DATA,
        dimension=DIMENSION,
        coords_s=COORDS_S,
        capacidad=CAPACIDAD
    )
    rs = p.imap(f, initial_candidates)
    for r in rs:
        bar.update(1)
        if r is not None:
            caminos.append(r)
            costos.append(calcular_distancias(r, COORDS))

  0%|          | 0/200 [00:00<?, ?it/s]

In [16]:
idx_minimo = costos.index(min(costos))
camino_minimo = caminos[idx_minimo]

print(idx_minimo)
print(cumple_capacidad(camino_minimo, DEMANDAS, CAPACIDAD))
print(calcular_distancias(camino_minimo, COORDS))

77
True
933431.0259261626


In [17]:
camino_minimo_d = [DATA[x-1] for x in camino_minimo]

In [18]:
def calcular_distancias_l(l):
    total = 0
    _, _, x0, y0 = l[0]
    for k1, dem1, x1, y1 in l[1:]:
        total += math.sqrt((x0 - x1) ** 2 + (y0 - y1) ** 2)
        x0, y0 = x1, y1
    _, _, xo, yo = l[0]
    _, _, xf, yf = l[-1]
    total += math.sqrt((xo - xf) ** 2 + (yo - yf) ** 2)
    return total

def cumple_capacidad_l(l, cap):
    capacidad_actual = 0
    for _, d, _, _ in l:
        capacidad_actual += d
        if capacidad_actual < 0 or capacidad_actual > cap:
            return False
    return True

In [19]:
%%timeit
calcular_distancias_l(camino_minimo_d)

7.05 ms ± 413 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%%timeit
cumple_capacidad_l(camino_minimo_d, CAPACIDAD)

1.72 ms ± 59.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
import time

path = camino_minimo_d

improvement = True
best_path = path
best_cost = calcular_distancias_l(best_path)
capacity_ok = cumple_capacidad_l(best_path, CAPACIDAD)

start = time.time()

with tqdm() as bar:
    while improvement:
        if time.time() - start > 60 * 10:
            break
        improvement = False
        for i in range(len(best_path) - 1):
            bar.set_description(f"Empiezo desde {i}")
            for k in range(i + 1, len(best_path)):
                bar.update()
                new_path = best_path[0:i]
                new_path.extend(reversed(best_path[i:k + 1]))
                new_path.extend(best_path[k + 1:])

                cumple_cap = cumple_capacidad_l(new_path, CAPACIDAD)
                if not cumple_cap:
                    continue

                new_cost = calcular_distancias_l(new_path)

                if new_cost >= best_cost:
                    continue
                else:
                    print(f"Improved from {best_cost} to {new_cost}")
                    best_cost = new_cost
                    best_path = new_path
                    improvement = True
                    break  # improvement found, return to the top of the while loop
            if improvement:
                break

0it [00:00, ?it/s]

Improved from 933431.0259261626 to 933428.6014689032
Improved from 933428.6014689032 to 933424.9474884081
Improved from 933424.9474884081 to 933421.585712655
Improved from 933421.585712655 to 933345.0459729485
Improved from 933345.0459729485 to 933335.9992363476
Improved from 933335.9992363476 to 933314.7468500973
Improved from 933314.7468500973 to 933199.8840161215
Improved from 933199.8840161215 to 933198.4090643357
Improved from 933198.4090643357 to 933189.908353961
Improved from 933189.908353961 to 933179.1683480118
Improved from 933179.1683480118 to 933172.8051694665
Improved from 933172.8051694665 to 933170.7061874529
Improved from 933170.7061874529 to 933169.3164151231
Improved from 933169.3164151231 to 933163.4213104999
Improved from 933163.4213104999 to 933160.7039759913


In [25]:
cumple_capacidad_l(best_path, CAPACIDAD)

True

In [26]:
calcular_distancias_l(best_path)

933160.7039759913

In [32]:
seq_str = " ".join(map(str, [x[0] for x in best_path]))

In [34]:
with open("entrega2_greedy_plus_2opt.txt", "w") as f:
    f.write(seq_str)